In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [16]:
path = '/Users/sanjayagrawal/Downloads/Kaggle/porto/'
train_data = pd.read_csv(path + 'train.csv')
test_data = pd.read_csv(path + 'test.csv')
sample_data = pd.read_csv(path + 'sample_submission.csv')
print (train_data.shape, test_data.shape, sample_data.shape)

(595212, 59) (892816, 58) (892816, 2)


## Data Split

In [ ]:
path_new = '/Users/sanjayagrawal/Downloads/Kaggle/porto/all_files/'
final_data = train_data

id  = pd.DataFrame({'id':list(set(final_data['id']))})
train = id.sample(frac=0.7,random_state=1)
valid = set(id['id'])-set(train['id'])
valid = pd.DataFrame({'id':list(valid)})
valid = valid.sample(frac=0.67,random_state=1)

test  = set(id['id'])-set(train['id'])-set(valid['id'])
test  = pd.DataFrame({'id':list(test)})

print(train.shape)
print(valid.shape)
print(test.shape)

train = train.merge(final_data, on=['id'], how='left')
valid = valid.merge(final_data, on=['id'], how='left')
test  = test.merge(final_data, on=['id'], how='left')

print(train.shape)
print(valid.shape)
print(test.shape)

In [ ]:
list = ['id', 'target']

train_x = train.drop(list, axis=1)
train_y = train['target']

valid_x = valid.drop(list, axis=1)
valid_y = valid['target']

test_x = test.drop(list, axis=1)
test_y = test['target']

In [21]:
from sklearn.model_selection import train_test_split
final_data1 = train_data
train_data_x = train.drop(['id', 'target'], axis=1)
train_data_y = train['target']
train_x, valid_x, train_y, valid_y = train_test_split(train_data_x, train_data_y, test_size=0.20, random_state=42)
print (train_x.shape, valid_x.shape, train_y.shape, valid_y.shape)

(333318, 57) (83330, 57) (333318,) (83330,)


## LightGBM model

In [22]:
model=lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.005, n_estimators=3000, 
                         max_bin=255, subsample_for_bin=50000, objective=None, min_split_gain=0.0, min_child_weight=5,
                         min_child_samples=10, subsample=1.0, subsample_freq=1, colsample_bytree=1.0, reg_alpha=0.0, 
                         reg_lambda=0.0, silent=True)

model.fit(train_x,train_y,
        eval_set=[(valid_x,valid_y)],
        eval_metric='logloss',
        early_stopping_rounds=30)

[1]	valid_0's binary_logloss: 0.688855
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's binary_logloss: 0.684606
[3]	valid_0's binary_logloss: 0.680399
[4]	valid_0's binary_logloss: 0.676233
[5]	valid_0's binary_logloss: 0.672108
[6]	valid_0's binary_logloss: 0.668023
[7]	valid_0's binary_logloss: 0.663979
[8]	valid_0's binary_logloss: 0.659973
[9]	valid_0's binary_logloss: 0.656007
[10]	valid_0's binary_logloss: 0.652078
[11]	valid_0's binary_logloss: 0.648187
[12]	valid_0's binary_logloss: 0.644333
[13]	valid_0's binary_logloss: 0.640516
[14]	valid_0's binary_logloss: 0.636735
[15]	valid_0's binary_logloss: 0.63299
[16]	valid_0's binary_logloss: 0.62928
[17]	valid_0's binary_logloss: 0.625605
[18]	valid_0's binary_logloss: 0.621964
[19]	valid_0's binary_logloss: 0.618357
[20]	valid_0's binary_logloss: 0.614783
[21]	valid_0's binary_logloss: 0.611242
[22]	valid_0's binary_logloss: 0.607734
[23]	valid_0's binary_logloss: 0.604258
[24]	valid_0's binary_logloss

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0,
        learning_rate=0.005, max_bin=255, max_depth=-1,
        min_child_samples=10, min_child_weight=5, min_split_gain=0.0,
        n_estimators=3000, nthread=-1, num_leaves=31, objective='binary',
        reg_alpha=0.0, reg_lambda=0.0, seed=0, silent=True, subsample=1.0,
        subsample_for_bin=50000, subsample_freq=1)

In [23]:
pred_valid =model.predict_proba(valid_x)
pred_test =model.predict_proba(test_x)

dt_valid=pd.DataFrame(pred_valid)
dt_test=pd.DataFrame(pred_test)

dt_valid.columns=['0_score','1_score']
dt_test.columns=['0_score','1_score']
print(dt_valid.shape, dt_test.shape)

(83330, 2) (58926, 2)


In [24]:
from sklearn.metrics import roc_curve, auc
fpr_valid,tpr_valid,thresholds_valid = roc_curve(valid_y,dt_valid['1_score'])
fpr_test,tpr_test,thresholds_test = roc_curve(test_y,dt_test['1_score'])

print ('valid:', 2*auc(fpr_valid,tpr_valid) - 1, 'test:', 2*auc(fpr_test,tpr_test) - 1)

valid: 0.261795905788 test: 0.285973915986


## Grid Search

In [ ]:
estimator = lgb.LGBMRegressor(num_leaves=31)

# model=lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.001, n_estimators=3000, 
#                          max_bin=255, subsample_for_bin=50000, objective=None, min_split_gain=0.0, min_child_weight=5,
#                          min_child_samples=10, subsample=1.0, subsample_freq=1, colsample_bytree=1.0, reg_alpha=0.0, 
#                          reg_lambda=0.0, silent=True)

param_grid = {
    'boosting_type' : ['gbdt'],
    'learning_rate': [0.005],
    'n_estimators': [1000, 500]
}

gbm = GridSearchCV(estimator, param_grid)

gbm.fit(train_x,train_y)

print('Best parameters found by grid search are:', gbm.best_params_)

In [ ]:
pred_valid =gbm.predict(valid_x)
pred_test =model.predict(test_x)

dt_valid=pd.DataFrame(pred_valid)
dt_test=pd.DataFrame(pred_test)

dt_valid.columns=['1_score']
dt_test.columns=['1_score']
print(dt_valid.shape, dt_test.shape)

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr_valid,tpr_valid,thresholds_valid = roc_curve(valid_y,dt_valid['1_score'])
fpr_test,tpr_test,thresholds_test = roc_curve(test_y,dt_test['1_score'])

print ('valid:', 2*auc(fpr_valid,tpr_valid) - 1, 'test:', 2*auc(fpr_test,tpr_test) - 1)

In [118]:
dt_valid.head()

,0
0,0.041768
1,0.083140
2,0.038555
3,0.070824
4,0.043463


## Test data Submission

In [10]:
final_x = test_data.drop('id', axis=1)
pred_final =model.predict_proba(final_x)
dt_final=pd.DataFrame(pred_final)
dt_final.columns=['0_score','target']

In [11]:
final_sub = test_data[['id']].join(dt_final['target'])
final_sub.to_csv(path_new + 'final_sub2.csv', index=False)

In [76]:
final_sub.head()

,id,target
0,0,0.028596
1,1,0.027498
2,2,0.028583
3,3,0.018398
4,4,0.038879
